In [4]:
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit, Aer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
import numpy as np

A two-local curcuit is a paramaterized circuit consisting of alternating rotation and entanglement layers

### Example 1: Basic Implementation ###

In [16]:
ex1 = TwoLocal(3, 'ry', 'cx', 'linear', reps=2, insert_barriers=True)
print(ex1)  # decompose the layers into standard gates

     ┌──────────┐ ░            ░ ┌──────────┐ ░            ░ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░───■────────░─┤ Ry(θ[3]) ├─░───■────────░─┤ Ry(θ[6]) ├
     ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░─┤ X ├──■───░─┤ Ry(θ[4]) ├─░─┤ X ├──■───░─┤ Ry(θ[7]) ├
     ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░──────┤ X ├─░─┤ Ry(θ[5]) ├─░──────┤ X ├─░─┤ Ry(θ[8]) ├
     └──────────┘ ░      └───┘ ░ └──────────┘ ░      └───┘ ░ └──────────┘


### Example 2: Multiple Rotations ###

In [28]:
ex2 = TwoLocal(3, ['ry','rz'], 'cz', 'full', reps=1, insert_barriers=True)
print(ex2)

     ┌──────────┐┌──────────┐ ░           ░ ┌──────────┐ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[3]) ├─░──■──■─────░─┤ Ry(θ[6]) ├─┤ Rz(θ[9]) ├
     ├──────────┤├──────────┤ ░  │  │     ░ ├──────────┤┌┴──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[4]) ├─░──■──┼──■──░─┤ Ry(θ[7]) ├┤ Rz(θ[10]) ├
     ├──────────┤├──────────┤ ░     │  │  ░ ├──────────┤├───────────┤
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[5]) ├─░─────■──■──░─┤ Ry(θ[8]) ├┤ Rz(θ[11]) ├
     └──────────┘└──────────┘ ░           ░ └──────────┘└───────────┘


### Example 3: Circular entanglement two ways ###

In [49]:
# 1 - default
ex3_1 = TwoLocal(3, 'x', 'crx', 'circular', reps=1)
print(ex3_1)

# 2 - using index pairs
entangler_map = [[0, 1], [1, 2], [2, 0]] 
ex3_2 = TwoLocal(3, 'x', 'crx', entangler_map, reps=1)
print(ex3_2)

     ┌───┐┌──────────┐               ┌───┐         
q_0: ┤ X ├┤ Rx(θ[0]) ├─────■─────────┤ X ├─────────
     ├───┤└────┬─────┘┌────┴─────┐   └───┘    ┌───┐
q_1: ┤ X ├─────┼──────┤ Rx(θ[1]) ├─────■──────┤ X ├
     ├───┤     │      └──────────┘┌────┴─────┐├───┤
q_2: ┤ X ├─────■──────────────────┤ Rx(θ[2]) ├┤ X ├
     └───┘                        └──────────┘└───┘
     ┌───┐                             ┌──────────┐┌───┐
q_0: ┤ X ├─────■───────────────────────┤ Rx(θ[2]) ├┤ X ├
     ├───┤┌────┴─────┐            ┌───┐└────┬─────┘└───┘
q_1: ┤ X ├┤ Rx(θ[0]) ├─────■──────┤ X ├─────┼───────────
     ├───┤└──────────┘┌────┴─────┐└───┘     │      ┌───┐
q_2: ┤ X ├────────────┤ Rx(θ[1]) ├──────────■──────┤ X ├
     └───┘            └──────────┘                 └───┘


### Example 4: Adding the Same TwoLocal Circuit To Itself ###

In [48]:
ex4_1 = TwoLocal(3,[], 'cry', 'linear', reps=1, insert_barriers=True)
print(ex4_1)
circuit = ex4_1 + ex4_1
print(circuit.decompose().draw()) 
# parameters (θ[0], θ[1], θ[2]) are the same

                              ░ 
q_0: ─────■───────────────────░─
     ┌────┴─────┐             ░ 
q_1: ┤ Ry(θ[0]) ├─────■───────░─
     └──────────┘┌────┴─────┐ ░ 
q_2: ────────────┤ Ry(θ[1]) ├─░─
                 └──────────┘ ░ 
                              ░                          ░ 
q_0: ─────■───────────────────░──────■───────────────────░─
     ┌────┴─────┐             ░ ┌────┴─────┐             ░ 
q_1: ┤ Ry(θ[0]) ├─────■───────░─┤ Ry(θ[0]) ├─────■───────░─
     └──────────┘┌────┴─────┐ ░ └──────────┘┌────┴─────┐ ░ 
q_2: ────────────┤ Ry(θ[1]) ├─░─────────────┤ Ry(θ[1]) ├─░─
                 └──────────┘ ░             └──────────┘ ░ 


### Example 5: Customizing Entanglement of Layers ###

In [ ]:
error1 = TwoLocal(3,[], 'cry', 'linear', reps=1, insert_barriers=True)
error2 = TwoLocal(3,[], 'cry', 'linear', reps=1, insert_barriers=True)
print(error1)
print(error2)
# circuit = error1 + error2 
# above line does not work, due to parameter names being the same within the same overall circuit for differing twolocal circuits
# must build in the layers altogether
layer_1 = [(0, 1), (0, 2)]
layer_2 = [(1, 2)]
layer_3 = [(0, 1), (1, 2), (2, 0)]
ex5 = TwoLocal(3, 'x', 'cry', [layer_1, layer_2, layer_3], reps=3, insert_barriers=True)
print(ex5)

### Adding a TwoLocal Circuit in to a Qiskit QuantumCircuit ###

In [ ]:
qc = QuantumCircuit(3)
tl1 = TwoLocal(3,'ry', 'cz', 'full', reps=1, insert_barriers=True)
qc += tl1
print(qc.decompose().draw())